# VacationPy
----

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

In [14]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.dropna()
cities_df.head()

,City_ID,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,esperance,-33.87,121.90,66.09,81,0,5.95,AU,1579969404
1,1,butaritari,3.07,172.79,84.49,74,0,10.09,KI,1579969404
2,2,bluff,-46.60,168.33,53.01,91,61,2.80,NZ,1579969388
3,3,talcahuano,-36.72,-73.12,61.23,56,0,12.84,CL,1579969405
4,4,castro,-24.79,-50.01,76.28,49,69,6.53,BR,1579969405


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
gmaps.configure(api_key=g_key)
locations = cities_df[["Lat", "lng"]]
humidity = cities_df["Humidity"].astype(float)

In [16]:
Heatmap = gmaps.figure(center = [0,0] ,zoom_level = 2)
 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=100,point_radius = 4)
Heatmap.add_layer(heat_layer)
Heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
temp_file=cities_df.loc[(cities_df['Max Temp']>=70)&(cities_df['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City_ID,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,75,esmeralda,21.85,-78.12,77.25,60,0,2.51,CU,1579969419
83,83,kahului,20.89,-156.47,70.00,78,0,5.01,US,1579969341
119,119,ballina,-28.87,153.57,73.99,99,0,10.00,AU,1579969430
126,126,constitucion,-35.33,-72.42,72.21,38,0,8.25,CL,1579969431
168,168,leeton,-34.57,146.40,71.04,31,0,3.74,AU,1579969442


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_name = requests.get(base_url, params=params)

    hotel_name = hotel_name.json()
    
    # Since some data may be missing adding try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

,City_ID,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,75,esmeralda,21.85,-78.12,77.25,60,0,2.51,CU,1579969419,Hotel Florida
1,83,kahului,20.89,-156.47,70.00,78,0,5.01,US,1579969341,"The Ritz-Carlton, Kapalua"
2,119,ballina,-28.87,153.57,73.99,99,0,10.00,AU,1579969430,Atlantic Byron Bay
3,126,constitucion,-35.33,-72.42,72.21,38,0,8.25,CL,1579969431,Hotel Playa El Cable
4,168,leeton,-34.57,146.40,71.04,31,0,3.74,AU,1579969442,The Griff Motel
5,189,tlalchapa,18.40,-100.47,73.22,49,0,1.12,MX,1579969444,Hotel Real de la Loma
6,215,puerto escondido,15.85,-97.07,75.69,62,0,0.78,MX,1579969435,Posada Real Puerto Escondido
7,243,torres,-29.34,-49.73,71.02,71,0,1.66,BR,1579969456,Mountain House Ecovillage Hotel
8,296,lazaro cardenas,17.96,-102.20,75.29,76,0,3.42,MX,1579969468,The Inn at Manzanillo Bay
9,331,cidreira,-30.18,-50.21,76.35,56,0,4.34,BR,1579969478,Hotel Ecovilly


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "lng"]]

In [21]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
Heatmap.add_layer(markers)
Heatmap

Figure(layout=FigureLayout(height='420px'))